Copyright (c) 2020, NVIDIA CORPORATION.
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at
    http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [1]:
import os, time
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
start = time.time()
very_start = time.time()

In [2]:
#import pandas as pd, 
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.max_rows', 500)
import cudf, cupy, time, rmm
cudf.__version__

'0.14.0a+3537.g4cafc98.dirty'

In [3]:
import dask as dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess

In [4]:
cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]
print(IPADDR)
# this is not the correct ip

169.254.8.236


In [5]:
cluster = LocalCUDACluster(ip='10.2.48.253')
client = Client(cluster)
client

Client Scheduler: tcp://10.2.48.253:41663 Dashboard: http://10.2.48.253:8787/status,Cluster Workers: 4 Cores: 4 Memory: 270.39 GB


In [6]:
#%%time
#client.run(cudf.set_allocator, pool=True, initial_pool_size=int(2**35)-int(2**32))
#client.run(cupy.cuda.set_allocator, rmm.rmm_cupy_allocator)

# Load Train

In [7]:
%%time
path = '/raid/data/recsys/train_split'
train = dask_cudf.read_parquet(f'{path}/train-preproc-fold-*.parquet')#,dtypes=dtypes)

CPU times: user 853 ms, sys: 427 ms, total: 1.28 s
Wall time: 1.27 s


In [8]:
%%time
# DROP UNUSED COLUMNS
cols_drop = ['links','hashtags0', 'hashtags1', 'fold']
train = train.drop(cols_drop,axis=1)

CPU times: user 63.9 ms, sys: 10.4 ms, total: 74.3 ms
Wall time: 66.4 ms


In [9]:
%%time
train, = dask.persist(train)
print(type(train), train.shape)

<class 'dask_cudf.core.DataFrame'> (Delayed('int-beb031f7-2d8c-4923-9dc0-7b2c47be50d5'), 25)
CPU times: user 16.5 ms, sys: 6.62 ms, total: 23.2 ms
Wall time: 21.3 ms


In [10]:
%%time
train = train.repartition(npartitions=4)
train, = dask.persist(train)
train.head()

CPU times: user 430 ms, sys: 68.2 ms, total: 499 ms
Wall time: 8.89 s


,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,...,b_account_creation,b_follows_a,reply,retweet,retweet_comment,like,engage_time,len_domains,len_hashtags,len_links
92560312,0,7,0,1,54,2020-02-09 09:26:50,0,14326,408,False,...,2018-03-13 13:47:49,True,0,0,0,0,1970-01-01 00:00:00,0,0,0
55533709,10,0,0,1,54,2020-02-09 18:41:35,10,237126,1193,True,...,2011-09-05 16:42:09,False,0,0,0,0,1970-01-01 00:00:00,0,0,0
37019981,20,5,0,1,3,2020-02-09 01:13:28,19,23079,1803,False,...,2016-05-19 02:19:01,False,0,0,0,0,1970-01-01 00:00:00,0,0,0
74047136,30,0,5,2,11,2020-02-07 12:15:20,29,769176,190,False,...,2019-09-10 09:17:08,False,0,0,0,1,2020-02-07 12:36:47,0,0,0
74047139,40,0,0,2,6,2020-02-08 14:14:39,35,73952,13,False,...,2019-12-11 15:38:45,False,0,0,0,1,2020-02-09 13:33:47,0,0,0


In [11]:
for i in range(4):
    print(i, len(train.partitions[i]))

0 29629913
1 44309248
2 29638018
3 44498059


In [12]:
label_names = ['reply', 'retweet', 'retweet_comment', 'like']
for col in train.columns:
    if col in label_names:
        train[col] = train[col].astype('float32')
    elif train[col].dtype=='int64':
        train[col] = train[col].astype('int32')
    elif train[col].dtype=='int16':
        train[col] = train[col].astype('int8')

In [13]:
%%time
train = train.reset_index(drop=True)

CPU times: user 61.9 ms, sys: 268 µs, total: 62.2 ms
Wall time: 58.3 ms


In [14]:
%%time
train, = dask.persist(train)
print(train.shape)

(Delayed('int-a8eef28e-0fe4-44f0-8080-36a07abe1271'), 25)
CPU times: user 54.9 ms, sys: 677 µs, total: 55.5 ms
Wall time: 53.4 ms


In [15]:
%%time 
# TIME FEATURES
# RAPIDS does this 5x faster than Pandas CPU
# If we didn't need to copy CPU to GPU to CPU, then 1300x faster!
def split_time(df):
    #gf = cudf.from_pandas(df[['timestamp']])
    df['dt_dow']  = df['timestamp'].dt.weekday#.to_array() 
    df['dt_hour'] = df['timestamp'].dt.hour#.to_array()
    df['dt_minute'] = df['timestamp'].dt.minute#.to_array()
    df['dt_second'] = df['timestamp'].dt.second#.to_array()
    return df

train = split_time(train)

CPU times: user 66.5 ms, sys: 3.8 ms, total: 70.3 ms
Wall time: 65.3 ms


In [16]:
train.head()[['engage_time','timestamp']]

,engage_time,timestamp
0,1970-01-01 00:00:00,2020-02-09 09:26:50
1,1970-01-01 00:00:00,2020-02-09 18:41:35
2,1970-01-01 00:00:00,2020-02-09 01:13:28
3,2020-02-07 12:36:47,2020-02-07 12:15:20
4,2020-02-09 13:33:47,2020-02-08 14:14:39


In [17]:
train.timestamp.dtype

dtype('<M8[ns]')

In [18]:
%%time
# ELAPSED TIME
for col in ['engage_time','timestamp']:
    train[col] = train[col].astype('int64')/1e3

CPU times: user 47 ms, sys: 646 µs, total: 47.7 ms
Wall time: 44.7 ms


In [19]:
%%time
train, = dask.persist(train)
print(type(train), train.shape)

<class 'dask_cudf.core.DataFrame'> (Delayed('int-e5dcc896-4aaf-4335-bbd4-139c9f5ecaa1'), 29)
CPU times: user 19.2 ms, sys: 3.09 ms, total: 22.3 ms
Wall time: 20.4 ms


In [20]:
train.head()[['engage_time','timestamp']]

,engage_time,timestamp
0,0.000000e+00,1.581240e+09
1,0.000000e+00,1.581274e+09
2,0.000000e+00,1.581211e+09
3,1.581079e+09,1.581078e+09
4,1.581255e+09,1.581171e+09


In [21]:
def set_nan(ds):
    mask = ds == 0
    ds.loc[mask] = np.nan
    return ds.nans_to_nulls()
train['engage_time'] = train['engage_time'].map_partitions(set_nan)

In [22]:
train['elapsed_time'] = train['engage_time'] - train['timestamp']
train['elapsed_time'] = train.elapsed_time.astype('float64')

In [23]:
print(train['elapsed_time'].min().compute(),train['elapsed_time'].max().compute())
print(train['elapsed_time'].mean().compute())

2.0 603956.0
15581.699535245267


In [24]:
%%time
train, = dask.persist(train)
print(type(train), train.shape)

<class 'dask_cudf.core.DataFrame'> (Delayed('int-7ffc6228-34d5-4292-9d8f-ac35561aa868'), 30)
CPU times: user 32.4 ms, sys: 0 ns, total: 32.4 ms
Wall time: 30.1 ms


In [25]:
train.head()

,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,...,like,engage_time,len_domains,len_hashtags,len_links,dt_dow,dt_hour,dt_minute,dt_second,elapsed_time
0,0,7,0,1,54,1.581240e+09,0,14326,408,False,...,0.0,null,0,0,0,6,9,26,50,null
1,10,0,0,1,54,1.581274e+09,10,237126,1193,True,...,0.0,null,0,0,0,6,18,41,35,null
2,20,5,0,1,3,1.581211e+09,19,23079,1803,False,...,0.0,null,0,0,0,6,1,13,28,null
3,30,0,5,2,11,1.581078e+09,29,769176,190,False,...,1.0,1.581079007e+09,0,0,0,4,12,15,20,1287.0
4,40,0,0,2,6,1.581171e+09,35,73952,13,False,...,1.0,1.581255227e+09,0,0,0,5,14,14,39,83948.0


# Feature Engineering 

In [26]:
%%time
# TRAIN FIRST 5 DAYS. VALIDATE LAST 2 DAYS
VALID_DOW = [1, 2]# order is [3, 4, 5, 6, 0, 1, 2]
valid = train[train['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)
train = train[~train['dt_dow'].isin(VALID_DOW)].reset_index(drop=True)

CPU times: user 78.1 ms, sys: 3.31 ms, total: 81.4 ms
Wall time: 75.9 ms


In [27]:
%%time
train,valid = dask.persist(train,valid)
print(type(train), train.shape, valid.shape)

<class 'dask_cudf.core.DataFrame'> (Delayed('int-1450cecc-05d4-419c-9636-33f44bd89aee'), 30) (Delayed('int-1523bcda-1447-41d4-8b2e-e11a2aa98159'), 30)
CPU times: user 27.8 ms, sys: 245 µs, total: 28 ms
Wall time: 24.8 ms


In [28]:
%%time
train = train.sort_values('timestamp').reset_index(drop=True)
valid = valid.sort_values('timestamp').reset_index(drop=True)
train,valid = dask.persist(train,valid)
train.head()

CPU times: user 1.11 s, sys: 72.2 ms, total: 1.18 s
Wall time: 6.15 s


,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,...,like,engage_time,len_domains,len_hashtags,len_links,dt_dow,dt_hour,dt_minute,dt_second,elapsed_time
0,164980,5,6625,2,3,1.580947e+09,112668,22984,6033,True,...,0.0,null,0,0,0,3,0,0,0,null
1,1173270,0,4,2,54,1.580947e+09,672136,5341,840,False,...,0.0,null,0,0,0,3,0,0,0,null
2,3240270,9,0,2,11,1.580947e+09,64906,256871,8,True,...,0.0,null,0,2,0,3,0,0,0,null
3,2468740,5,4,2,54,1.580947e+09,1278002,16289,376,False,...,0.0,null,0,0,0,3,0,0,0,null
4,3653210,0,0,2,11,1.580947e+09,98180,9841,6,True,...,0.0,null,0,5,0,3,0,0,0,null


In [29]:
for i in range(4):
    print(i,len(train.partitions[i]))

0 26885321
1 26832264
2 26822864
3 26696073


In [30]:
for i in range(4):
    print(i,len(valid.partitions[i]))

0 10236930
1 10267563
2 10191750
3 10142473


### Target Encode

In [31]:
class MTE_one_shot:
    
    def __init__(self, folds, smooth, seed=42, mode='gpu'):
        self.folds = folds
        self.seed = seed
        self.smooth = smooth
        if mode=='gpu':
            self.np = cupy
            self.df = cudf
        else:
            self.np = np
            self.df = pd
        self.mode = mode
        
    def fit_transform(self, train, x_col, y_col, y_mean=None, out_col = None, out_dtype=None):
        
        self.y_col = y_col
        self.np.random.seed(self.seed)
        
        if 'fold' not in train.columns:
            fsize = len(train)//self.folds
            if isinstance(train,dask_cudf.core.DataFrame):
                #train['fold'] = train.map_partitions(lambda cudf_df: cudf_df.index%self.folds)
                train['fold'] = 1
                train['fold'] = train['fold'].cumsum()
                train['fold'] = train['fold']//fsize
                train['fold'] = train['fold']%self.folds
            else:
                #train['fold'] = self.np.random.randint(0,self.folds,len(train))
                train['fold'] = (train.index.values//fsize)%self.folds
        
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'TE_{tag}_{self.y_col}'
        
        if y_mean is None:
            y_mean = train[y_col].mean()#.compute().astype('float32')
        self.mean = y_mean
        
        cols = ['fold',x_col] if isinstance(x_col,str) else ['fold']+x_col
        
        agg_each_fold = train.groupby(cols).agg({y_col:['count','sum']}).reset_index()
        agg_each_fold.columns = cols + ['count_y','sum_y']
        
        agg_all = agg_each_fold.groupby(x_col).agg({'count_y':'sum','sum_y':'sum'}).reset_index()
        cols = [x_col] if isinstance(x_col,str) else x_col
        agg_all.columns = cols + ['count_y_all','sum_y_all']
        
        agg_each_fold = agg_each_fold.merge(agg_all,on=x_col,how='left')
        agg_each_fold['count_y_all'] = agg_each_fold['count_y_all'] - agg_each_fold['count_y']
        agg_each_fold['sum_y_all'] = agg_each_fold['sum_y_all'] - agg_each_fold['sum_y']
        agg_each_fold[out_col] = (agg_each_fold['sum_y_all']+self.smooth*self.mean)/(agg_each_fold['count_y_all']+self.smooth)
        agg_each_fold = agg_each_fold.drop(['count_y_all','count_y','sum_y_all','sum_y'],axis=1)
        
        agg_all[out_col] = (agg_all['sum_y_all']+self.smooth*self.mean)/(agg_all['count_y_all']+self.smooth)
        agg_all = agg_all.drop(['count_y_all','sum_y_all'],axis=1)
        self.agg_all = agg_all
        
        train.columns
        cols = ['fold',x_col] if isinstance(x_col,str) else ['fold']+x_col
        train = train.merge(agg_each_fold,on=cols,how='left')
        del agg_each_fold
        #self.agg_each_fold = agg_each_fold
        if self.mode=='gpu':
            if isinstance(train,dask_cudf.core.DataFrame):
                train[out_col] = train.map_partitions(lambda cudf_df: cudf_df[out_col].nans_to_nulls())
            else:
                train[out_col] = train[out_col].nans_to_nulls()
        train[out_col] = train[out_col].fillna(self.mean)
        
        if out_dtype is not None:
            train[out_col] = train[out_col].astype(out_dtype)
        return train
    
    def transform(self, test, x_col, out_col = None, out_dtype=None):
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'TE_{tag}_{self.y_col}'
        test = test.merge(self.agg_all,on=x_col,how='left')
        test[out_col] = test[out_col].fillna(self.mean)
        if out_dtype is not None:
            test[out_col] = test[out_col].astype(out_dtype)
        return test
 

In [32]:
%%time
# cuDF TE ENCODING IS SUPER FAST!!
idx = 0; cols = []
start = time.time()
for t in ['reply', 'retweet', 'retweet_comment', 'like']:
    start = time.time()
    for c in ['media', 'tweet_type', 'language', 'a_user_id', 'b_user_id']:
        out_col = f'TE_{c}_{t}'
        encoder = MTE_one_shot(folds=5,smooth=20)
        train = encoder.fit_transform(train, c, t, out_col=out_col, out_dtype='float32')
        valid = encoder.transform(valid, c, out_col=out_col, out_dtype='float32')
        cols.append(out_col)
        del encoder
        train,valid = dask.persist(train,valid)
        train.head()

CPU times: user 18 s, sys: 1.26 s, total: 19.3 s
Wall time: 1min 29s


In [33]:
train['fold'].value_counts().compute()

0    21447306
1    21447304
2    21447304
3    21447304
4    21447304
Name: fold, dtype: int64

### Multiple Column Target Encode

In [34]:
%%time
# cuDF TE ENCODING IS SUPER FAST!!
idx = 0; cols=[]
c = ['domains','language','b_follows_a','tweet_type','media','a_is_verified']
for t in ['reply', 'retweet', 'retweet_comment', 'like']:
    out_col = f'TE_multi_{t}'
    encoder = MTE_one_shot(folds=5,smooth=20)
    train = encoder.fit_transform(train, c, t, out_col=out_col, out_dtype='float32')
    valid = encoder.transform(valid, c, out_col=out_col, out_dtype='float32')
    cols.append(out_col)
    del encoder

CPU times: user 2.61 s, sys: 96.4 ms, total: 2.71 s
Wall time: 2.6 s


In [35]:
%%time
train,valid = dask.persist(train,valid)
train.head()

CPU times: user 1.58 s, sys: 130 ms, total: 1.71 s
Wall time: 8.92 s


,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,...,TE_b_user_id_retweet_comment,TE_media_like,TE_tweet_type_like,TE_language_like,TE_a_user_id_like,TE_b_user_id_like,TE_multi_reply,TE_multi_retweet,TE_multi_retweet_comment,TE_multi_like
0,62920,5,0,2,54,1.580948e+09,781,562432,327,False,...,0.007708,0.494331,0.532514,0.449791,0.790396,0.448912,0.027371,0.102288,0.007343,0.603071
1,62920,5,0,2,54,1.580948e+09,781,562432,327,False,...,0.006167,0.494331,0.532514,0.449791,0.790396,0.439130,0.027371,0.102288,0.007343,0.603071
2,62920,5,0,2,54,1.580948e+09,781,555910,325,False,...,0.007341,0.494331,0.532514,0.449791,0.790396,0.427535,0.027371,0.102288,0.007343,0.603071
3,62920,5,0,2,54,1.580948e+09,781,562432,327,False,...,0.005316,0.494331,0.532514,0.449791,0.790396,0.482008,0.027371,0.102288,0.007343,0.603071
4,62920,5,0,2,54,1.580948e+09,781,562432,327,False,...,0.006167,0.494331,0.532514,0.449791,0.790396,0.479130,0.027371,0.102288,0.007343,0.603071


### Elapsed Time Target Encode

In [36]:
%%time
# cuDF TE ENCODING IS SUPER FAST!!
idx = 0; cols = []
for c in ['media', 'tweet_type', 'language']:#, 'a_user_id', 'b_user_id']:
    for t in ['elapsed_time']:
        out_col = f'TE_{c}_{t}'
        encoder = MTE_one_shot(folds=5,smooth=20)
        train = encoder.fit_transform(train, c, t, out_col=out_col)
        out_dtype='float32' #if 'user_id' in c else None
        valid = encoder.transform(valid, c, out_col=out_col, out_dtype=out_dtype)
        cols.append(out_col)
        #del encoder

CPU times: user 1.4 s, sys: 54.2 ms, total: 1.45 s
Wall time: 1.38 s


In [37]:
%%time
train,valid = dask.persist(train,valid)
train.head()

CPU times: user 909 ms, sys: 41.1 ms, total: 950 ms
Wall time: 5.47 s


,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,...,TE_language_like,TE_a_user_id_like,TE_b_user_id_like,TE_multi_reply,TE_multi_retweet,TE_multi_retweet_comment,TE_multi_like,TE_media_elapsed_time,TE_tweet_type_elapsed_time,TE_language_elapsed_time
0,1518712,7,0,1,3,1.580948e+09,1324,6103201,375,True,...,0.411003,0.458515,0.390358,0.006305,0.031647,0.000697,0.185066,17537.0768,11601.091716,19749.286736
1,1518712,7,0,1,3,1.580948e+09,1324,6071996,376,True,...,0.411003,0.458515,0.433837,0.006305,0.031647,0.000697,0.185066,17537.0768,11601.091716,19749.286736
2,1518712,7,0,1,3,1.580948e+09,1324,6103201,375,True,...,0.411003,0.458515,0.408102,0.006305,0.031647,0.000697,0.185066,17537.0768,11601.091716,19749.286736
3,1518712,7,0,1,3,1.580948e+09,1324,6103201,375,True,...,0.411003,0.458515,0.427535,0.006305,0.031647,0.000697,0.185066,17537.0768,11601.091716,19749.286736
4,1518712,7,0,1,3,1.580948e+09,1324,6103201,375,True,...,0.411003,0.458515,0.453557,0.006305,0.031647,0.000697,0.185066,17537.0768,11601.091716,19749.286736


In [38]:
valid.head()

,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id,a_follower_count,a_following_count,a_is_verified,...,TE_language_like,TE_a_user_id_like,TE_b_user_id_like,TE_multi_reply,TE_multi_retweet,TE_multi_retweet_comment,TE_multi_like,TE_media_elapsed_time,TE_tweet_type_elapsed_time,TE_language_elapsed_time
0,61737,5,0,2,50,1.581379e+09,2824,31861000,80,True,...,0.423782,0.414891,0.635704,0.016941,0.036669,0.004081,0.472134,18000.140625,19014.623047,17206.958984
1,61737,5,0,2,50,1.581379e+09,2824,31861000,80,True,...,0.423782,0.414891,0.480676,0.016941,0.036669,0.004081,0.472134,18000.140625,19014.623047,17206.958984
2,61737,5,0,2,50,1.581379e+09,2824,31852634,80,True,...,0.423782,0.414891,0.479130,0.016941,0.036669,0.004081,0.472134,18000.140625,19014.623047,17206.958984
3,61737,5,0,2,50,1.581379e+09,2824,31861000,80,True,...,0.423782,0.414891,0.587595,0.016941,0.036669,0.004081,0.472134,18000.140625,19014.623047,17206.958984
4,61737,5,0,2,50,1.581379e+09,2824,31852634,80,True,...,0.423782,0.414891,0.477315,0.016941,0.036669,0.004081,0.472134,18000.140625,19014.623047,17206.958984


### Count Encode

In [39]:
class FrequencyEncoder:
    
    def __init__(self, seed=42, mode='gpu'):
        self.seed = seed
        if mode=='gpu':
            self.np = cupy
            self.df = cudf
        else:
            self.np = np
            self.df = pd
        self.mode = mode
        
    def fit_transform(self, train, x_col, c_col=None, out_col = None):
        self.np.random.seed(self.seed)
        if c_col is None or c_col not in train.columns:
            c_col = 'dummy'
            train[c_col] = 1
            drop = True
        else:
            drop = False
            
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'CE_{tag}_norm'
            
        cols = [x_col] if isinstance(x_col,str) else x_col
        agg_all = train.groupby(cols).agg({c_col:'count'}).reset_index()
        if drop:
            train = train.drop(c_col,axis=1)
        agg_all.columns = cols + [out_col]
        agg_all[out_col] = agg_all[out_col].astype('int32')
        agg_all[out_col] = agg_all[out_col]*1.0/len(train)
        agg_all[out_col] = agg_all[out_col].astype('float32')
    
        train = train.merge(agg_all,on=cols,how='left')
        del agg_all
        #print(train.columns)
        if self.mode=='gpu':
            if isinstance(train,dask_cudf.core.DataFrame):
                train[out_col] = train.map_partitions(lambda cudf_df: cudf_df[out_col].nans_to_nulls())
            else:
                train[out_col] = train[out_col].nans_to_nulls()
        return train
    
    def transform(self, test, x_col, c_col=None, out_col = None):
        return self.fit_transform(test, x_col, c_col, out_col)
 

In [40]:
class CountEncoder:
    
    def __init__(self, seed=42, mode='gpu'):
        self.seed = seed
        if mode=='gpu':
            self.np = cupy
            self.df = cudf
        else:
            self.np = np
            self.df = pd
        self.mode = mode
        
    def fit_transform(self, train, test, x_col, out_col = None):
        self.np.random.seed(self.seed)
        
        common_cols = [i for i in train.columns if i in test.columns and i!=x_col]

        if len(common_cols):
            c_col = common_cols[0]
            drop = False
        else:
            c_col = 'dummy'
            train[c_col] = 1
            test[c_col]=1
            drop = True
            
        if out_col is None:
            tag = x_col if isinstance(x_col,str) else '_'.join(x_col)
            out_col = f'CE_{tag}_norm'
            
        cols = [x_col] if isinstance(x_col,str) else x_col
        agg_all = train.groupby(cols).agg({c_col:'count'}).reset_index()
        agg_all.columns = cols + [out_col]
        
        agg_test = test.groupby(cols).agg({c_col:'count'}).reset_index()
        agg_test.columns = cols + [out_col+'_test']
        agg_all = agg_all.merge(agg_test,on=cols,how='left')
        agg_all[out_col+'_test'] = agg_all[out_col+'_test'].fillna(0)
        agg_all[out_col] = agg_all[out_col] + agg_all[out_col+'_test']
        agg_all = agg_all.drop(out_col+'_test', axis=1)
        del agg_test
            
        if drop:
            train = train.drop(c_col,axis=1)
            test = test.drop(c_col,axis=1)
        train = train.merge(agg_all,on=cols,how='left')
        test = test.merge(agg_all,on=cols,how='left')
        del agg_all
        return train,test

In [41]:
%%time
# cuDF CE ENCODING IS SUPER FAST!!
idx = 0; cols = []
for c in ['media', 'tweet_type', 'language', 'a_user_id', 'b_user_id']:
    encoder = CountEncoder()
    out_col = f'CE_{c}'
    train,valid = encoder.fit_transform(train, valid, c, out_col=out_col)
    print
    del encoder
    train,valid = dask.persist(train,valid)
    train.head()

CPU times: user 1.82 s, sys: 187 ms, total: 2 s
Wall time: 7.48 s


In [42]:
%%time
# cuDF CE ENCODING IS SUPER FAST!!
idx = 0; cols = []
for c in ['media', 'tweet_type', 'language', 'a_user_id', 'b_user_id']:
    encoder = FrequencyEncoder()
    out_col = f'CE_{c}_norm'
    train = encoder.fit_transform(train, c, c_col='tweet_id', out_col=out_col)
    valid = encoder.transform(valid, c, c_col='tweet_id', out_col=out_col)
    cols.append(out_col)
    del encoder
    train,valid = dask.persist(train,valid)
    train.head()

CPU times: user 2.77 s, sys: 252 ms, total: 3.02 s
Wall time: 9.22 s


### Difference Encode (Lag Features)

In [43]:
def diff_encode_cudf_v1(train,col,tar,sft=1):
    train[col+'_sft'] = train[col].shift(sft)
    train[tar+'_sft'] = train[tar].shift(sft)
    out_col = f'DE_{col}_{tar}_{sft}'
    train[out_col] = train[tar]-train[tar+'_sft']
    mask = '__MASK__'
    train[mask] = train[col] == train[col+'_sft']
    train = train.drop([col+'_sft',tar+'_sft'],axis=1)
    train[out_col] = train[out_col]*train[mask]
    train = train.drop(mask,axis=1)
    return train

In [44]:
%%time

# cuDF DE ENCODING IS FAST!!
idx = 0; cols = []; sc = 'timestamp'
for c in ['b_user_id']:
    for t in ['b_follower_count','b_following_count','language']:
        for s in [1,-1]:
            start = time.time()
            train = diff_encode_cudf_v1(train, col=c, tar=t, sft=s)
            valid = diff_encode_cudf_v1(valid, col=c, tar=t, sft=s)
            train,valid = dask.persist(train,valid)
            train.head()
            end = time.time(); idx += 1
            print('DE',c,t,s,'%.1f seconds'%(end-start))

DE b_user_id b_follower_count 1 1.1 seconds
DE b_user_id b_follower_count -1 1.2 seconds
DE b_user_id b_following_count 1 1.2 seconds
DE b_user_id b_following_count -1 1.0 seconds
DE b_user_id language 1 1.3 seconds
DE b_user_id language -1 1.2 seconds
CPU times: user 4.32 s, sys: 400 ms, total: 4.72 s
Wall time: 7.09 s


### Diff Language

In [45]:
train_lang = train[['a_user_id', 'language', 'tweet_id']].drop_duplicates()
valid_lang = valid[['a_user_id', 'language', 'tweet_id']].drop_duplicates()
train_lang_count = train_lang.groupby(['a_user_id', 'language']).agg({'tweet_id':'count'}).reset_index()
valid_lang_count = valid_lang.groupby(['a_user_id', 'language']).agg({'tweet_id':'count'}).reset_index()
train_lang_count,valid_lang_count = dask.persist(train_lang_count,valid_lang_count)
train_lang_count.head()
del train_lang,valid_lang

In [46]:
%%time
train_lang_count = train_lang_count.merge(valid_lang_count,on=['a_user_id', 'language'],how='left')
train_lang_count['tweet_id_y'] = train_lang_count['tweet_id_y'].fillna(0)
train_lang_count['tweet_id_x'] = train_lang_count['tweet_id_x'] + train_lang_count['tweet_id_y']
train_lang_count = train_lang_count.drop('tweet_id_y',axis=1)
train_lang_count.columns = ['a_user_id', 'top_language', 'language_count']
train_lang_count, = dask.persist(train_lang_count)
train_lang_count.head()

CPU times: user 90.7 ms, sys: 1.84 ms, total: 92.5 ms
Wall time: 301 ms


,a_user_id,top_language,language_count
0,20499,18,2
1,20499,38,2
2,20499,44,37
3,20499,47,5
4,20499,54,44


In [47]:
%%time
train_lang_count = train_lang_count.sort_values(['a_user_id', 'language_count'])
train_lang_count['a_user_shifted'] = train_lang_count['a_user_id'].shift(1)
train_lang_count = train_lang_count[train_lang_count['a_user_id']!=train_lang_count['a_user_shifted']]
train_lang_count = train_lang_count.drop(['a_user_shifted','language_count'],axis=1)
train_lang_count.columns = ['a_user_id','top_language']
train_lang_count, = dask.persist(train_lang_count)
train_lang_count.head()

CPU times: user 75.1 ms, sys: 8.69 ms, total: 83.8 ms
Wall time: 150 ms


,a_user_id,top_language
70964,0,4
70966,1,47
70951,2,21
70954,3,25
70967,4,54


In [48]:
def diff_language(df,df_lang_count):
    df = df.merge(df_lang_count,how='left', left_on='b_user_id', right_on='a_user_id')
    df['nan_language'] = df['top_language'].isnull()
    df['same_language'] = df['language'] == df['top_language']
    df['diff_language'] = df['language'] != df['top_language']
    df['same_language'] = df['same_language']*(1-df['nan_language'])
    df['diff_language'] = df['diff_language']*(1-df['nan_language'])
    df = df.drop('top_language',axis=1)
    return df

In [49]:
%%time
train = diff_language(train,train_lang_count)
valid = diff_language(valid,train_lang_count)
train,valid = dask.persist(train,valid)
train.head()

CPU times: user 768 ms, sys: 86.3 ms, total: 854 ms
Wall time: 2.2 s


,tweet_id,media,domains,tweet_type,language,timestamp,a_user_id_x,a_follower_count,a_following_count,a_is_verified,...,DE_b_user_id_b_follower_count_1,DE_b_user_id_b_follower_count_-1,DE_b_user_id_b_following_count_1,DE_b_user_id_b_following_count_-1,DE_b_user_id_language_1,DE_b_user_id_language_-1,a_user_id_y,nan_language,same_language,diff_language
0,117178,0,0,0,11,1.580948e+09,86219,2963357,156,True,...,0,0,0,0,0,0,null,True,0,0
1,117178,0,0,0,11,1.580948e+09,86219,2963357,156,True,...,0,0,0,0,0,0,null,True,0,0
2,117178,0,0,0,11,1.580948e+09,86219,2957303,156,True,...,0,0,0,0,0,0,null,True,0,0
3,117178,0,0,0,11,1.580948e+09,86219,2963357,156,True,...,0,0,0,0,0,0,null,True,0,0
4,742378,0,0,2,11,1.580948e+09,86219,2963357,156,True,...,0,0,0,0,0,0,null,True,0,0


## Rate feature

In [50]:
%%time
# follow rate feature
train['a_ff_rate'] = (train['a_following_count'] / train['a_follower_count']).astype('float32')
train['b_ff_rate'] = (train['b_follower_count']  / train['b_following_count']).astype('float32')
valid['a_ff_rate']  = (valid['a_following_count'] / valid['a_follower_count']).astype('float32')
valid['b_ff_rate']  = (valid['b_follower_count']  / valid['b_following_count']).astype('float32')

CPU times: user 137 ms, sys: 4.33 ms, total: 141 ms
Wall time: 133 ms


In [51]:
train,valid = dask.persist(train,valid)

In [52]:
train.head()
valid.head()
print()

# Summarize Features

In [53]:
%%time

label_names = ['reply', 'retweet', 'retweet_comment', 'like']
DONT_USE = ['timestamp','a_account_creation','b_account_creation','engage_time',
            'fold','b_user_id','a_user_id', 'dt_dow',
            'a_account_creation', 'b_account_creation', 'elapsed_time',
             'links','domains','hashtags0','hashtags1']
DONT_USE += label_names
features = [c for c in train.columns if c not in DONT_USE]

RMV = [c for c in DONT_USE if c in train.columns and c not in label_names]
RMV

CPU times: user 4.02 ms, sys: 13 µs, total: 4.04 ms
Wall time: 3.89 ms


['timestamp',
 'a_account_creation',
 'b_account_creation',
 'engage_time',
 'fold',
 'b_user_id',
 'dt_dow',
 'a_account_creation',
 'b_account_creation',
 'elapsed_time',
 'domains']

In [54]:
%%time

for col in RMV:
    #print(col, col in train.columns)
    if col in train.columns:
        train = train.drop(col,axis=1)
        train, = dask.persist(train)
        train.head()
        

CPU times: user 939 ms, sys: 185 ms, total: 1.12 s
Wall time: 1.28 s


In [55]:
%%time

for col in RMV:
    #print(col, col in valid.columns)
    if col in valid.columns:
        valid = valid.drop(col,axis=1)
        valid, = dask.persist(valid,)
        valid.head()
        

CPU times: user 801 ms, sys: 119 ms, total: 920 ms
Wall time: 1.07 s


# Train Model Validate
We will train on random 10% of first 5 days and validation on last 2 days

In [56]:
%%time

SAMPLE_RATIO = 0.1
SEED = 1

if SAMPLE_RATIO < 1.0:
    train['sample'] = train['tweet_id'].map_partitions(lambda cudf_df: cudf_df.hash_encode(stop=10))
    print(len(train))
    
    train = train[train['sample']<10*SAMPLE_RATIO]
    train, = dask.persist(train)
    train.head()
    print(len(train))


Y_train = train[label_names]
Y_train, = dask.persist(Y_train)
Y_train.head()    
    
train = train.drop(['sample','tweet_id']+label_names,axis=1)
train, = dask.persist(train)
train.head()


features = [c for c in train.columns if c not in DONT_USE]
print('Using %i features:'%(len(features)),train.shape[1])
np.asarray(features)

107236522
10716577
Using 66 features: 66
CPU times: user 275 ms, sys: 35.2 ms, total: 310 ms
Wall time: 588 ms


array(['media', 'tweet_type', 'language', 'a_user_id_x',
       'a_follower_count', 'a_following_count', 'a_is_verified',
       'b_follower_count', 'b_following_count', 'b_is_verified',
       'b_follows_a', 'len_domains', 'len_hashtags', 'len_links',
       'dt_hour', 'dt_minute', 'dt_second', 'TE_media_reply',
       'TE_tweet_type_reply', 'TE_language_reply', 'TE_a_user_id_reply',
       'TE_b_user_id_reply', 'TE_media_retweet', 'TE_tweet_type_retweet',
       'TE_language_retweet', 'TE_a_user_id_retweet',
       'TE_b_user_id_retweet', 'TE_media_retweet_comment',
       'TE_tweet_type_retweet_comment', 'TE_language_retweet_comment',
       'TE_a_user_id_retweet_comment', 'TE_b_user_id_retweet_comment',
       'TE_media_like', 'TE_tweet_type_like', 'TE_language_like',
       'TE_a_user_id_like', 'TE_b_user_id_like', 'TE_multi_reply',
       'TE_multi_retweet', 'TE_multi_retweet_comment', 'TE_multi_like',
       'TE_media_elapsed_time', 'TE_tweet_type_elapsed_time',
       'TE_langu

In [57]:
SAMPLE_RATIO = 0.35 # VAL SET NOW SIZE OF TEST SET
SEED = 1
if SAMPLE_RATIO < 1.0:
    print(len(valid))
    valid['sample'] = valid['tweet_id'].map_partitions(lambda cudf_df: cudf_df.hash_encode(stop=10))
    
    valid = valid[valid['sample']<10*SAMPLE_RATIO]
    valid, = dask.persist(valid)
    valid.head()
    print(len(valid))
    
Y_valid = valid[label_names]
Y_valid, = dask.persist(Y_valid)
Y_valid.head()

valid = valid.drop(['sample','tweet_id']+label_names,axis=1)
valid, = dask.persist(valid)
valid.head()

40838716
16288926


,media,tweet_type,language,a_user_id_x,a_follower_count,a_following_count,a_is_verified,b_follower_count,b_following_count,b_is_verified,...,DE_b_user_id_b_following_count_1,DE_b_user_id_b_following_count_-1,DE_b_user_id_language_1,DE_b_user_id_language_-1,a_user_id_y,nan_language,same_language,diff_language,a_ff_rate,b_ff_rate
33,5,2,54,1579,34907345,243,True,10,140,False,...,0,0,0,0,964466,False,1,0,0.000007,0.071429
34,5,2,54,1579,34907345,243,True,513,728,False,...,0,0,0,0,8936009,False,0,1,0.000007,0.704670
35,5,2,54,1579,34907345,243,True,15,217,False,...,0,0,0,0,null,True,0,0,0.000007,0.069124
36,5,2,54,1579,34907345,243,True,87,221,False,...,0,0,0,0,null,True,0,0,0.000007,0.393665
37,5,2,54,1579,34907345,243,True,3,70,False,...,0,0,0,0,null,True,0,0,0.000007,0.042857


In [58]:
import xgboost as xgb
print('XGB Version',xgb.__version__)

xgb_parms = { 
    'max_depth':8, 
    'learning_rate':0.1, 
    'subsample':0.8,
    'colsample_bytree':0.3, 
    'eval_metric':'logloss',
    'objective':'binary:logistic',
    'tree_method':'gpu_hist',
    'predictor' : 'gpu_predictor'
}


XGB Version 1.2.0-SNAPSHOT


In [59]:
if train.columns.duplicated().sum()>0:
    raise Exception(f'duplicated!: { train.columns[train.columns.duplicated()] }')
print('no dup :) ')
print(f'X_train.shape {train.shape}')
print(f'X_valid.shape {valid.shape}')

no dup :) 
X_train.shape (Delayed('int-a33f43d7-84ea-407c-aa6d-be1f5f7e1b97'), 66)
X_valid.shape (Delayed('int-5d45c190-cacd-49c9-9462-07b90197567e'), 66)


In [60]:
%%time

for col in train.columns:
    if train[col].dtype=='bool':
        train[col] = train[col].astype('int8')
        valid[col] = valid[col].astype('int8')
train,valid = dask.persist(train,valid)
train.head()
valid.head()

CPU times: user 444 ms, sys: 47.5 ms, total: 492 ms
Wall time: 1.62 s


,media,tweet_type,language,a_user_id_x,a_follower_count,a_following_count,a_is_verified,b_follower_count,b_following_count,b_is_verified,...,DE_b_user_id_b_following_count_1,DE_b_user_id_b_following_count_-1,DE_b_user_id_language_1,DE_b_user_id_language_-1,a_user_id_y,nan_language,same_language,diff_language,a_ff_rate,b_ff_rate
33,5,2,54,1579,34907345,243,1,10,140,0,...,0,0,0,0,964466,0,1,0,0.000007,0.071429
34,5,2,54,1579,34907345,243,1,513,728,0,...,0,0,0,0,8936009,0,0,1,0.000007,0.704670
35,5,2,54,1579,34907345,243,1,15,217,0,...,0,0,0,0,null,1,0,0,0.000007,0.069124
36,5,2,54,1579,34907345,243,1,87,221,0,...,0,0,0,0,null,1,0,0,0.000007,0.393665
37,5,2,54,1579,34907345,243,1,3,70,0,...,0,0,0,0,null,1,0,0,0.000007,0.042857


In [61]:
%%time
# TRAIN AND VALIDATE

NROUND = 300
VERBOSE_EVAL = 50
#ESR = 50
    
oof = np.zeros((len(valid),len(label_names)))
preds = []
for i in range(4):

    name = label_names[i]
    print('#'*25);print('###',name);print('#'*25)
       
    start = time.time(); print('Creating DMatrix...')
    dtrain = xgb.dask.DaskDMatrix(client,data=train,label=Y_train.iloc[:, i])
    dvalid = xgb.dask.DaskDMatrix(client,data=valid,label=Y_valid.iloc[:, i])
    print('Took %.1f seconds'%(time.time()-start))
             
    start = time.time(); print('Training...')
    model = xgb.dask.train(client, xgb_parms, 
                           dtrain=dtrain,
                           #evals=[(dtrain,'train'),(dvalid,'valid')],
                           num_boost_round=NROUND,
                           #early_stopping_rounds=ESR,
                           verbose_eval=VERBOSE_EVAL) 
    print('Took %.1f seconds'%(time.time()-start))
        
    start = time.time(); print('Predicting...')
    #Y_valid[f'pred_{name}'] = xgb.dask.predict(client,model,valid)
    #oof[:, i] += xgb.dask.predict(client,model,dvalid).compute()
    preds.append(xgb.dask.predict(client,model,valid))
    print('Took %.1f seconds'%(time.time()-start))
        
    del model, dtrain, dvalid

#########################
### reply
#########################
Creating DMatrix...
Took 0.1 seconds
Training...
Took 13.8 seconds
Predicting...


/home/jiwei/anaconda3/envs/rapids/lib/python3.7/site-packages/distributed/worker.py:3349: UserWarning: Large object of size 4.57 MB detected in task graph: 
  [<function predict.<locals>.mapped_predict at 0x7f ... titions>, True]
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


Took 0.5 seconds
#########################
### retweet
#########################
Creating DMatrix...
Took 0.1 seconds
Training...
Took 13.5 seconds
Predicting...
Took 0.6 seconds
#########################
### retweet_comment
#########################
Creating DMatrix...
Took 0.1 seconds
Training...
Took 12.9 seconds
Predicting...
Took 0.5 seconds
#########################
### like
#########################
Creating DMatrix...
Took 0.1 seconds
Training...
Took 13.6 seconds
Predicting...
Took 0.6 seconds
CPU times: user 4.09 s, sys: 581 ms, total: 4.67 s
Wall time: 56.2 s


In [62]:
for i in preds:
    print(i.isnull().sum().compute())

0
0
0
0


In [63]:
yvalid = Y_valid[label_names].values.compute()
yvalid = cupy.asnumpy(yvalid)

oof = cupy.array([i.values.compute() for i in preds]).T
oof = cupy.asnumpy(oof)

# Compute Validation Metrics

In [64]:
from sklearn.metrics import precision_recall_curve, auc, log_loss

def compute_prauc(pred, gt):
  prec, recall, thresh = precision_recall_curve(gt, pred)
  prauc = auc(recall, prec)
  return prauc

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

# FAST METRIC FROM GIBA
def compute_rce_fast(pred, gt):
    cross_entropy = log_loss(gt, pred)
    yt = np.mean(gt)     
    strawman_cross_entropy = -(yt*np.log(yt) + (1 - yt)*np.log(1 - yt))
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

In [65]:
%%time
txt = ''
for i in range(4):
    prauc = compute_prauc(oof[:,i], yvalid[:, i])
    rce   = compute_rce_fast(oof[:,i], yvalid[:, i])
    txt_ = f"{label_names[i]:20} PRAUC:{prauc:.5f} RCE:{rce:.5f}"
    print(txt_)
    txt += txt_ + '\n'

reply                PRAUC:0.18020 RCE:19.69112
retweet              PRAUC:0.53612 RCE:29.32064
retweet_comment      PRAUC:0.05623 RCE:11.93086
like                 PRAUC:0.78253 RCE:26.83609
CPU times: user 36.6 s, sys: 2.58 s, total: 39.1 s
Wall time: 37.4 s


In [66]:
print('This notebook took %.1f minutes'%((time.time()-very_start)/60.))

This notebook took 4.5 minutes
